# Lista 9 - BERT

Nessa lista exploraremos como usar o HuggingFace para aplicar modelos baseados em transformers. O HuggingFace é (também) um conjunto de bibliotecas para Python que abstrai o treinamento, ajuste fino e utilização de modelos baseados em transformers, como o BERT. Como complemento aos slides das aulas, use o próprio [curso/livro texto do HuggingFace](https://huggingface.co/learn/nlp-course/chapter1/1), disponível gratuitamente, e a documentação das funções, para a resolução desta lista.

Vamos começar instalando as bibliotecas do ecosistema da HuggingFace que serão necessários e importando os itens necessários (Na dúvida sobre o que usar, dê uma olhada no que está sendo importado).

In [7]:
!pip install transformers[torch] tokenizers datasets
!pip install accelerate -U

zsh:1: no matches found: transformers[torch]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, pipeline, DataCollatorWithPadding, TrainingArguments, Trainer
from datasets import load_dataset, load_metric
import torch
import numpy as np

# Questão 1 - Usando um modelo ajustado
Nessa questão, vamos explorar como utilizar um modelo BERT que já foi ajustado para uma tarefa específica, rodando-o num input ou conjunto de inputs de interesse.
Usaremos o modelo **BERT Go Emotion Base** cujo identificador no HuggingFace é ``bhadresh-savani/bert-base-go-emotion``. Esse é um modelo BERT ajustado para o problema de classificar um texto em inglês segundo as emoções que ele expressa.
Complete a célula abaixo de forma a importar e usar o modelo em questão para a classificação do texto armazenado na variável ``text``, usando a estrutura ``pipeline``.

In [11]:
text="Then plainly know my heart's dear love is set on the fair daughter of rich Capulet: as mine on hers, so hers is set on mine;"
# Instancie o tokenizador do modelo
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Instancie o modelo
model = AutoModelForSequenceClassification.from_pretrained("bhadresh-savani/bert-base-go-emotion")

# Instancie o pipeline de uso a partir do modelo e do tokenizador
# Obs: instancie de forma a retornar o score previsto para todas as classes e não apenas
# a classe mais provável
recognizer = pipeline("text-classification", model=model, tokenizer=tokenizer, top_k=None)

#Aplique o modelo ao texto
recognizer(text)



[[{'label': 'neutral', 'score': 0.6134440898895264},
  {'label': 'realization', 'score': 0.09147773683071136},
  {'label': 'approval', 'score': 0.06752301007509232},
  {'label': 'annoyance', 'score': 0.03160139173269272},
  {'label': 'disappointment', 'score': 0.022592734545469284},
  {'label': 'optimism', 'score': 0.02241162583231926},
  {'label': 'surprise', 'score': 0.02102230302989483},
  {'label': 'amusement', 'score': 0.01915157213807106},
  {'label': 'desire', 'score': 0.011300671845674515},
  {'label': 'admiration', 'score': 0.011197982355952263},
  {'label': 'disapproval', 'score': 0.010413268581032753},
  {'label': 'sadness', 'score': 0.008721446618437767},
  {'label': 'excitement', 'score': 0.007849073968827724},
  {'label': 'disgust', 'score': 0.007495712023228407},
  {'label': 'embarrassment', 'score': 0.0074385362677276134},
  {'label': 'fear', 'score': 0.006812149193137884},
  {'label': 'joy', 'score': 0.006175273098051548},
  {'label': 'nervousness', 'score': 0.00553304

# Questão 2 - Ajustando o BERTimbau para uma tarefa de classificação
O [BERTimbau](https://huggingface.co/neuralmind/bert-base-portuguese-cased) é o modelo de atenção pioneiro para o Português. Ele é uma versão do BERT pré-treinado com dados em português e está disponível abertamente no HUggingFace em dois tamanhos. Nesse exercício vamos fazer o ajuste-fino no BERTimbau Base (já pré-treinado) para a tarefa de Inferência em Linguagem Natural. Essa é a tarefa de receber dois textos, uma Premissa e uma Hipótese e classificar qual é a relação lógica entre premissa e hipótese. Usaremos o dataset ASSIN2 que modela esse como um problema de classificação em duas classes: Entailment (Existe acarretamento lógico) e None (Não existe acarretamento lógico).

Complete o código a seguir nos locais indicados, de forma a ajustar o BERTimbau para a tarefa citada. Na dúvida, consulte a documentação das referidas classes e o livro-texto do HuggingFace.

In [8]:
dataset = load_dataset("assin2")

dataset = dataset.rename_columns({"entailment_judgment": "label"})

tokenizer = AutoTokenizer.from_pretrained(
    "neuralmind/bert-base-portuguese-cased", do_lower_case=False, model_max_length=512
)


# TOKENIZAÇÃO:
# Crio uma função de tokenização dessa que processe cada dataset dessa forma permite que usemos a função .map() para aplicá-la no próprio dataset.
# O resultado de uma aplicação com map é um objeto do tipo dataset então continuaremos dispondo das comodidades dessa estrutura de dados,
# o que não aconteceríamos se aplicássemos o tokenizador diretamente ao dataset, caso no qual o resultado retornado seria um dicionário comum.
def tokenize_function(examples):
    return tokenizer(
        [z for z in zip(examples["premise"], examples["hypothesis"])], truncation=True
    )


tokenized_datasets = dataset.map(tokenize_function, batched=True)

dataset

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence_pair_id', 'premise', 'hypothesis', 'relatedness_score', 'label'],
        num_rows: 6500
    })
    test: Dataset({
        features: ['sentence_pair_id', 'premise', 'hypothesis', 'relatedness_score', 'label'],
        num_rows: 2448
    })
    validation: Dataset({
        features: ['sentence_pair_id', 'premise', 'hypothesis', 'relatedness_score', 'label'],
        num_rows: 500
    })
})

In [9]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence_pair_id', 'premise', 'hypothesis', 'relatedness_score', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6500
    })
    test: Dataset({
        features: ['sentence_pair_id', 'premise', 'hypothesis', 'relatedness_score', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2448
    })
    validation: Dataset({
        features: ['sentence_pair_id', 'premise', 'hypothesis', 'relatedness_score', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 500
    })
})

In [10]:

torch.manual_seed(1562517)

#CARREGAMENTO E PREPARAÇÃO DO DATASET

#Carregue o dataset "assin2"
dataset = load_dataset("assin2")

dataset=dataset.rename_columns({'entailment_judgment':'label'})


#CARREGAMENTO DO TOKENIZADOR
#Carregue o tokenizador do modelo (use  model_max_length=512):
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False, model_max_length=512)

#TOKENIZAÇÃO:
# Crio uma função de tokenização dessa que processe cada dataset dessa forma permite que usemos a função .map() para aplicá-la no próprio dataset.
# O resultado de uma aplicação com map é um objeto do tipo dataset então continuaremos dispondo das comodidades dessa estrutura de dados,
# o que não aconteceríamos se aplicássemos o tokenizador diretamente ao dataset, caso no qual o resultado retornado seria um dicionário comum.
def tokenize_function(examples):
  return tokenizer([z for z in zip(examples["premise"], examples["hypothesis"])], truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

#"COLETOR" DE DADOS:
# Instancie um DataCollatorWithPadding a partir do tokenizador importado
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#PARTICIONAMENTO E AMOSTRAGEM DO DATASET:
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
eval_dataset = tokenized_datasets["validation"].shuffle(seed=42)
test_set = tokenized_datasets["test"].shuffle(seed=42)

# CONFIGURAÇÕES DO MODELO:
label2id = {'NONE': 0, 'ENTAILMENT':1}
id2label = {0: 'NONE', 1: 'ENTAILMENT'}
config= AutoConfig.from_pretrained("neuralmind/bert-base-portuguese-cased", label2id=label2id, id2label=id2label, num_labels=2, seed=1)

# CARREGAMENTO DO MODELO:
# Carregue o modelo BERTimbau, usando as configurações definidas na linha anterior
model = AutoModelForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased')

#ARGUMENTOS DE TREINAMENTO:
# Usando a classe TrainingArgs, crie um objeto que estabeleça os seguintes argumentos de treinamento?
# 1 época de treinamento
# 1 Validação a cada época
# 1 salvamento do modelo intermediário por época

training_args = TrainingArguments(
    output_dir="model_dir",
    learning_rate=2e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)


# MÉTRICA DE AVALIAÇÃO:

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# TREINADOR (Abstrai o loop de treinamento):
# Crie um objeto Trainer para treinar o modelo carregado, usando os argumentos
# de treinamento estabelecidos, os conjuntos de treinamento e validação produzidos
# a função compute_metrics para computar a métrica e o coletor de dados previamente instanciado

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# TREINAMENTO E SALVAMENTO
trained=trainer.train()
trainer.save_model("Bertinho")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/125 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.43933045864105225, 'eval_accuracy': 0.81, 'eval_runtime': 3.8942, 'eval_samples_per_second': 128.397, 'eval_steps_per_second': 16.178, 'epoch': 1.0}
{'train_runtime': 54.7304, 'train_samples_per_second': 18.271, 'train_steps_per_second': 2.284, 'train_loss': 0.58202099609375, 'epoch': 1.0}


# Questão 3
Ao carregar o modelo, na questão anterior, você deve ter recebido um aviso que começa com *Some weights of BertForSequenceClassification were not initialized...*
Explique porque carregar o modelo gera esse aviso e porque ele é esperado.



O aviso nos lembra que os pesos devem ser estimados antes de usar o modelo para inferências, e é esperado pois os pesos não são inicializados ao carregar o modelo. Os pesos são calculados posteriormente no treino `Trainer`, com os argumentos definidos em `training_args`. 